In [ ]:
# pip install GDAL

In [1]:
import pandas as pd
import numpy as np

from osgeo import gdal

# get our latitude and longitude data
df = pd.read_csv('Data/latitude_longitude.csv')

In [2]:
df.head()

,location,location_type,country,province,county,city,latitude,longitude
0,Argentina-Buenos_Aires,province,Argentina,Buenos Aires,NaN,NaN,-34.603684,-58.381559
1,Argentina-CABA,province,Argentina,Ciudad de Buenos Aires,NaN,NaN,-34.603684,-58.381559
2,Argentina-Cordoba,province,Argentina,Cordoba,NaN,NaN,-31.420083,-64.188776
3,Argentina-Entre_Rios,province,Argentina,Entre Rios,NaN,NaN,-31.774665,-60.495646
4,Argentina-Santa_Fe,province,Argentina,Santa Fe,NaN,NaN,-31.610658,-60.697294


In [3]:
# get the geoTIF map
ds = gdal.Open('Data/gpw-v4-population-density_2015.tif')

In [4]:
print(ds)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x1251be720> >


In [5]:
rows = ds.RasterYSize
cols = ds.RasterXSize

transform = ds.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = transform[5]

In [6]:
band = ds.GetRasterBand(1)

In [7]:
def get_pop_density(latitude, longitude, 
                           xOrigin=xOrigin, yOrigin=yOrigin,
                           pixelWidth=pixelWidth, pixelHeight=pixelHeight,
                           band=band):
    x = longitude
    y = latitude

    # This reads three pixels in x- and y- direction
    try:
        # Subtract one off the end because I want to read 3 x 3 region
        size = 100

        dist_matrix = np.meshgrid(np.arange(-size, size+1), 
                                  np.arange(-size, size+1))
        dist_matrix = np.sqrt((dist_matrix[0]**2 + dist_matrix[1]**2))
        sort_order = dist_matrix.ravel().argsort()

        xOffset = int((x - xOrigin) / pixelWidth) - size
        yOffset = int((y - yOrigin) / pixelHeight) - size

        data = band.ReadAsArray(xOffset, yOffset, 2*size+1, 2*size+1)
        data_sort = data.ravel()[sort_order]

        density = data_sort[data_sort>0][:9].mean()
    except:
        density = np.NaN

    return density



In [8]:
df['density_per_km'] = df.apply(lambda x: get_pop_density(x.latitude, x.longitude), axis=1)

In [10]:
df = df[['location','density_per_km']]

In [11]:
# save csv file for future needs
df.to_csv('population_density.csv', encoding='utf-8', index=False)